In [0]:
%run ./setAccess

In [0]:
%run ./loadBase

In [0]:
# passed by pipeline (user defined) parameter (set for specific database)
configFileName = dbutils.widgets.get("configFileName")  

# passed by pipeline (trigger) parameters
fileName = dbutils.widgets.get("fileName")
filePath = dbutils.widgets.get("filePath") 

# load cleaned file from 'cleansed' layer
data = (spark.read.format("parquet")
        .option("header", "true")
        .option("inferSchema", "true")
        .load("abfss://"+container+"@"+storageAccountName+".dfs.core.windows.net/"+
              dirs["lvl2"]+fileName.split(".")[0]+".parquet")
       )

# load config for databases information
config = (spark.read
          .format("json")
          .option("multiline","true")
          .load("abfss://"+container+"@"+storageAccountName+".dfs.core.windows.net/"+
                dirs["config"]+configFileName)
          .collect()
         )
databases = list(set([conf.asDict()["targetBase"] for conf in config]))

# update and load dimension tables
dimTables = []
for database in databases:
    dimTables.append(updateDimensionTable(config,data,database))
        
# update and load facts table
factsTable = updateFactsTable(config,data,databases,dimTables,configFileName)